In [ ]:
!pip install langchain
!pip install pinecone-client
!pip install pypdf
!pip install openai
!pip install tiktoken
!pip install faiss-cpu
!pip install unstructured

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 404.4/404.4 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/

In [ ]:
pip install -U langchain-community


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.9 MB/s eta 0:00:00


In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import openai
from langchain.vectorstores import Pinecone
from langchain.chains import retrieval_qa
from langchain.prompts import PromptTemplate
#from langchain.vectorstores import FAISS
#from langchain.chains.question_answering import load_qa_chain
#from langchain.llms import OpenAI
import os

In [ ]:
# creating a folder named pdfs
!mkdir pdfs

Now, I will upload all the pdfs or text data into pdfs folder

In [ ]:
PyPDFDirectoryLoader("pdfs")

Able to load the file

In [ ]:
loader = PyPDFDirectoryLoader("pdfs")

In [ ]:
data = loader.load()

In [ ]:
data[0]

Document(metadata={'source': 'pdfs/Digress.pdf', 'page': 0}, page_content='Published as a conference paper at ICLR 2023\nDIGRESS : D ISCRETE DENOISING DIFFUSION FOR\nGRAPH GENERATION\nCl´ement Vignac∗\nLTS4, EPFL\nLausanne, SwitzerlandIgor Krawczuk∗\nLIONS, EPFL\nLausanne, SwitzerlandAntoine Siraudin\nLTS4, EPFL\nLausanne, Switzerland\nBohan Wang\nLTS4, EPFL\nLausanne, SwitzerlandVolkan Cevher\nLIONS, EPFL\nLausanne, SwitzerlandPascal Frossard\nLTS4, EPFL\nLausanne, Switzerland\nABSTRACT\nThis work introduces DiGress, a discrete denoising diffusion model for generat-\ning graphs with categorical node and edge attributes. Our model utilizes a discrete\ndiffusion process that progressively edits graphs with noise, through the process\nof adding or removing edges and changing the categories. A graph transformer\nnetwork is trained to revert this process, simplifying the problem of distribution\nlearning over graphs into a sequence of node and edge classification tasks. We\nfurther improve

In [ ]:
len(data)

22

Perform Tokenisation

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)

In [ ]:
text_chunks = text_splitter.split_documents(data)

In [ ]:
text_chunks[0]

Document(metadata={'source': 'pdfs/Digress.pdf', 'page': 0}, page_content='Published as a conference paper at ICLR 2023\nDIGRESS : D ISCRETE DENOISING DIFFUSION FOR\nGRAPH GENERATION\nCl´ement Vignac∗\nLTS4, EPFL\nLausanne, SwitzerlandIgor Krawczuk∗\nLIONS, EPFL\nLausanne, SwitzerlandAntoine Siraudin\nLTS4, EPFL\nLausanne, Switzerland\nBohan Wang\nLTS4, EPFL\nLausanne, SwitzerlandVolkan Cevher\nLIONS, EPFL\nLausanne, SwitzerlandPascal Frossard\nLTS4, EPFL\nLausanne, Switzerland\nABSTRACT\nThis work introduces DiGress, a discrete denoising diffusion model for generat-')

In [ ]:
print(text_chunks[0])

page_content='Published as a conference paper at ICLR 2023
DIGRESS : D ISCRETE DENOISING DIFFUSION FOR
GRAPH GENERATION
Cl´ement Vignac∗
LTS4, EPFL
Lausanne, SwitzerlandIgor Krawczuk∗
LIONS, EPFL
Lausanne, SwitzerlandAntoine Siraudin
LTS4, EPFL
Lausanne, Switzerland
Bohan Wang
LTS4, EPFL
Lausanne, SwitzerlandVolkan Cevher
LIONS, EPFL
Lausanne, SwitzerlandPascal Frossard
LTS4, EPFL
Lausanne, Switzerland
ABSTRACT
This work introduces DiGress, a discrete denoising diffusion model for generat-' metadata={'source': 'pdfs/Digress.pdf', 'page': 0}


In [ ]:
print(len(text_chunks))

150


In [ ]:
print(text_chunks[0].page_content)

Published as a conference paper at ICLR 2023
DIGRESS : D ISCRETE DENOISING DIFFUSION FOR
GRAPH GENERATION
Cl´ement Vignac∗
LTS4, EPFL
Lausanne, SwitzerlandIgor Krawczuk∗
LIONS, EPFL
Lausanne, SwitzerlandAntoine Siraudin
LTS4, EPFL
Lausanne, Switzerland
Bohan Wang
LTS4, EPFL
Lausanne, SwitzerlandVolkan Cevher
LIONS, EPFL
Lausanne, SwitzerlandPascal Frossard
LTS4, EPFL
Lausanne, Switzerland
ABSTRACT
This work introduces DiGress, a discrete denoising diffusion model for generat-


In [ ]:
print(text_chunks[1].page_content)

ing graphs with categorical node and edge attributes. Our model utilizes a discrete
diffusion process that progressively edits graphs with noise, through the process
of adding or removing edges and changing the categories. A graph transformer
network is trained to revert this process, simplifying the problem of distribution
learning over graphs into a sequence of node and edge classification tasks. We
further improve sample quality by introducing a Markovian noise model that pre-


In [ ]:
print(text_chunks[2].page_content)

serves the marginal distribution of node and edge types during diffusion, and by
incorporating auxiliary graph-theoretic features. A procedure for conditioning the
generation on graph-level features is also proposed. DiGress achieves state-of-the-
art performance on molecular and non-molecular datasets, with up to 3x validity
improvement on a planar graph dataset. It is also the first model to scale to the
large GuacaMol dataset containing 1.3M drug-like molecules without the use of


In [ ]:
print(text_chunks[149].page_content)

Published as a conference paper at ICLR 2023
Figure 10: Non curated samples generated by DiGress, trained on QM9 with implicit hydrogens
(top), and explicit hydrogens (bottom).
Figure 11: Non curated samples generated by Guacamol (top) and Moses (bottom). While there
are some failure cases (disconnected molecules or invalid molecules), our model is the first non
autoregressive method that scales to these datasets that are much more complex than the standard
QM9.
22


In [ ]:
print(text_chunks[150].page_content) #since indexing starts from 0, since len is 150 (0 to 149)

IndexError: list index out of range

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "{Enter here the key}"

In [ ]:
#create an object of OpenAI Embedding
embeddings = OpenAIEmbeddings()

In [ ]:
embeddings.embed_query("How are you") # it created the embedding
# this is giving an error because we dont have purchased the OpenAI embedding
# NOTE: Dimensions of word2vec embedding is 300

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [ ]:
len(embeddings.embed_query("Hello world"))
# o/p: 1536, this is the openAI embedding dimension length

To obtain the Pinecone API key and environment information, follow these steps:

**Sign Up / Log In:**

Visit the Pinecone website and either sign up for a new account or log in if you already have one.
Access the API Key:

Once logged in, go to the API Keys section of the dashboard.
There, you will see an option to create a new API key. Copy the key and keep it secure, as you’ll need it to access Pinecone services.
Find the Environment Name:

In the same dashboard, look for the Environment section. It will display your environment name (often formatted like us-west1-gcp).
Note this down, as it will be required to initialize the Pinecone client along with the API key.

In [ ]:
# Import pinecone and store all the embedding in the Pinecone

PINECONE_API_KEY = os.environ.get('PINECONE_API_KEY', '{Enter here PINECONE_API_KEY}')
PINECONE_API_ENV = os.environ.get('PINECONE_API_ENV', '{Enter here PINECONE_API_ENV}')

In [ ]:
import pinecone

In [ ]:
pinecone.init(api_key=PINECONE_API_KEY, #find at app.pinecone.io
              environment=PINECONE_API_ENV  #next to api key in console
              )

In [ ]:
index_name = "testing"  # put in the name of your pinecone index here

In [ ]:
index = pinecone.Index(index_name) # index = pinecone.Index("testing")

# Create Embeddings for each of the Text Chunk

In [ ]:
# once you run this command you can see all the embeddings stored in the pinecone interface
docsearch = Pinecone.from_texts([t.page_content for t in text_chunks], embeddings, index_name=index_name) #for whatever chunks we have created we will be creating the embeddings

In [ ]:
docsearch # in this object we have all the vectors

In [ ]:
docsearch.as_retriever()  # gives info about Database, which Embeddings used, and where are they stored

In [ ]:
query = "what does  Scaffold similarity measures"

In [ ]:
docs = docsearch.similarity_search(query) # performing similarity search

In [ ]:
docs  #but we will get the output in the form of vectors that is numbers

In [ ]:
# Now converting number to string, we will need an LLM

In [ ]:
llm = OpenAI()    # initialising an LLM

In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()

**1. RetrievalQA:**

RetrievalQA is a type of chain in LangChain used for question-answering (QA) tasks. It works by retrieving relevant documents from a knowledge base (like a vector store) and then applying a language model (LLM) to generate answers based on the retrieved documents.


**2. from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever()):**

**llm=llm:**

This specifies the language model to use for generating answers. In your case, llm is likely an instance of a large language model (like OpenAI's GPT) that will process the retrieved information and formulate the final answer.

**chain_type="stuff":**

This specifies the type of chain to use for combining the retrieved documents and generating the final answer. In this case, "stuff" refers to the simplest chain type in LangChain, where the retrieved documents are "stuffed" into a single prompt before being passed to the LLM for answering.

**retriever=docsearch.as_retriever():**

This specifies the retriever to use for fetching relevant documents. docsearch is likely an instance of a vector store (e.g., Pinecone, FAISS) where the documents have been indexed based on their embeddings. The method .as_retriever() converts the vector store into a retriever interface that LangChain can use to pull relevant documents based on the user's question.

**Summary of What It Does:**
This statement creates a question-answering chain using:

A language model (llm) to generate answers.

A document retriever (docsearch.as_retriever()) to fetch relevant documents based on the question.

The stuff chain type, which combines all the retrieved documents into a single input to the language model.

The resulting qa chain can take a user's question, retrieve relevant documents from the vector store, and generate a detailed answer using the language model.

In [ ]:
qa.run(query) # gives answer in string format from the PDF

# QA

In [ ]:
import sys
while True:
  user_input = input(f"Input Prompt: ")
  if user_input == "exit":
    print('Exiting....')
    sys.exit()
  if user_input == "":
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")